In [2]:
!pip install wordcloud

     |████████████████████████████████| 438 kB 22.4 MB/s eta 0:00:01
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np
import pandas as pd
import konlpy

In [2]:
df = pd.read_csv('News_s20230601.csv',dtype= {'press_id': 'object', 'section_id': 'object'})

In [3]:
# 정규표현식으로 한글만 필터링
import re

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    result = hangul.sub('', text)
    return result

df['content'] = df['content'].apply(lambda x : text_cleaning(x))


In [4]:
re.compile('[^ ㄱ-ㅣ가-힣]+').sub('',df['content'][0])

'오신환 서울시 전 정무부시장이 일 서울시의 경계경보 발령 논란에 대해 오발령 공지를 한 행정안전부를 겨냥해 그럴 여유가 있었다면 서울시와 좀 더 소통해서 서울시가 직접 정정 또는 해제하도록 권고하는 게 오히려 바람직한 결과를 낳았을 것이라고 비판했다'

In [5]:
kkma = konlpy.tag.Kkma() #형태소 분석기 꼬꼬마(Kkma)

nouns = df['content'][:50].apply(kkma.nouns)
nouns

0     [오신, 오신환, 환, 서울시, 전, 정무, 정무부시, 부시, 일, 경계, 경계경보...
1     [부산, 벡스코, 국제, 국제해양방위산업, 해양, 방위, 산업, 전은, 정부, 세계...
2     [인천시, 중구, 동구, 통합, 제물, 제물포구, 포구, 신설, 영종, 영종도, 도...
3     [종, 국방부, 장관, 일, 국회, 열린, 국방위, 전체, 전체회의, 회의, 북한,...
4     [유, 유승민, 승, 민, 전, 의원, 일, 사회, 사회관계망서비스, 관계, 망, ...
5     [합동, 합동참모본부, 참모, 본부, 일, 어청도, 서방, 여, 해상, 인양, 북,...
6     [종, 국방부, 장관, 일, 국회, 국방, 국방위원회, 위원회, 전체, 전체회의, ...
7     [김, 김기현, 기현, 국민, 힘, 대표, 일, 경기, 경기도, 도, 수원, 경기도...
8     [국민, 힘, 급, 당직자, 씨, 강원, 강원도, 강원도당, 도, 당, 근무, 직원...
9     [해, 해사안전위원회, 사, 안전, 위원회, 일, 영국, 런던, 본부, 제차, 회의...
10    [민주당, 탈당, 무소속, 부천, 부천시의회, 시의회, 박성호, 의원, 일, 임시회...
11    [지난달, 일, 북한, 평안북, 평안북도, 도, 철산군, 동창, 새, 새발사장, 발...
12    [국방, 국방과학연구소국과, 과학, 연구소, 국과, 연, 흥, 종합, 종합시험장, ...
13    [이재명, 민주당, 대표, 일, 사회, 사회관계망서비스, 관계, 망, 서비스, 경찰...
14    [조, 조현동, 현동, 주미, 한국, 한국대사, 대사, 일, 일현지시간, 현지, 시...
15    [일, 오전, 해군, 해경, 국방부, 직할, 국군, 국군화생방방호사령부, 화생방, ...
16    [국회, 국방, 국방위원회, 위원회, 일, 오전, 시, 전체, 전체회의, 회의, 북...
17    [김, 북한, 국무, 국무위원장, 위원장, 여동생, 김여정, 여정, 노동당,

In [6]:
nouns = nouns.explode()
nouns

0      오신
0     오신환
0       환
0     서울시
0       전
     ... 
49      핵
49    핵합의
49     합의
49     복원
49     전망
Name: content, Length: 1661, dtype: object

In [23]:
df_word['word']

0      오신
0     오신환
0       환
0     서울시
0       전
     ... 
49      핵
49    핵합의
49     합의
49     복원
49     전망
Name: word, Length: 1661, dtype: object

In [7]:
df_word = pd.DataFrame({'word' : nouns})
df_word['count'] = df_word['word'].str.len()
#df_word = df_word.query('count >= 2')
#df_word.sort_values(by= ['count'], ascending = False)

In [57]:
df_word = df_word.groupby('word', as_index = False).count().sort_values(by= ['count'], ascending = False)
df_word

,word,count
412,발사,12
392,미국,12
483,북한,12
640,시간,9
159,국민,8
...,...,...
505,비금면,1
504,비금,1
503,비교,1
502,비공개,1


In [83]:
#불용어제거
df_word.sort_values(by = ['count'], ascending = False)

okt = Okt()
stop_words = sw

result =[]
for i in range(len(df_word['word'])):
    txt = df_word['word'][i]
    stop_words = set(stop_words)
    word_tokens = okt.morphs(txt)
    result += [word for word in word_tokens if not word in stop_words]

result
df_word = df_word[(df_word['word'].isin(result))]

In [87]:
df_word.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 412 to 502
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    1023 non-null   object
 1   count   1023 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.0+ KB


## 참고

In [88]:
# dic_word = df_word.set_index('word').to_dict()['count']
# dic_word

In [89]:
# sw = pd.read_csv('stopwords.csv',encoding= 'utf-8')
# sw = sw['불용어']
# sw.head

In [90]:
# okt = Okt()

# example = "휴 아이고 아이쿠 아홉"
# stop_words = sw

# stop_words = set(stop_words)
# word_tokens = okt.morphs(example)

# result = [word for word in word_tokens if not word in stop_words]

# print('불용어 제거 전 :',word_tokens) 
# print('불용어 제거 후 :',result)

In [8]:
# wc = WordCloud(random_state = 123, font_path = '/usr/share/fonts/truetype.ttf', width = 400,height = 400, background_color = 'white')

# img_wordcloud = wc.generate_from_frequencies(dic_word)

# plt.figure(figsize = (10, 10)) # 크기 지정하기
# plt.axis('off') # 축 없애기
# #plt.imshow(img_wordcloud) # 결과 보여주기


In [4]:
import pandas as pd
d = pd.read_csv('News_20230601.csv',dtype= {'press_id': 'object', 'section_id': 'object'})

In [5]:
d[d['headline'] =='“尹대통령과 커피 마시는 꿈 꾸고 대박”…복권 1등 당첨된 여성 사연 보니']

,id,headline,writer,url,content,created_date,press_id,section_id
453,459,“尹대통령과 커피 마시는 꿈 꾸고 대박”…복권 1등 당첨된 여성 사연 보니,김수연 기자,https://n.news.naver.com/article/022/000381865...,‘스피또 1000’ 제71회 복권 당첨 사연 공개돼 동행복권 홈페이지 갈무리 ...,2023-06-01 11:21:07,022,102


In [14]:
d['content'][453]

'‘스피또 1000’ 제71회 복권 당첨 사연 공개돼    동행복권 홈페이지 갈무리    \xa0  윤석열 대통령과 커피를 마시는 꿈을 꾼 뒤 복권 1등에 당첨됐다는 여성의 사연이 공개됐다.  \xa0  1일 동행복권에 따르면 경기 용인시의 한 복권판매점에서 ‘스피또 1000’ 제71회 복권을 구입한 A씨가\xa0최근 해당 복권 1등에 당첨돼 5억원을 수령했다.  \xa0  A씨는 “평소 재미 삼아 로또와 스피또 복권을 구매해왔다”며 “최근 윤 대통령과 커피 마시는 꿈을 꿨고, 그 기운을 받아 당첨복권 12장을 교환하러 집 주변에 있는 복권판매점에 방문했다”고 운을 뗐다.  \xa0  그는 “처음 방문한 판매점에서는 스피또1000 복권이 소진돼서 교환할 수 없었다”며 “두 번째로 간 판매점에 10장이 남아 있어서 10장만 교환했다. 세 번째 판매점에서 남은 2장을 교환하고 집으로 돌아왔다”고 설명했다.  \xa0  이어 “두 번째 판매점에서 교환한 10장을 다 긁고 마지막 남은 2장을 긁었는데, 5억원이 당첨됐다”며 “남편은 농담하지 말라고 하더라. 처음에는 얼떨떨하고 믿기지 않았는데, 고객센터에 당첨 확인과 방문 예약을 하니 실감이 났다”고 전했다.  \xa0  그러면서 “최근 스피또1000 2등에도 당첨돼서 올해 기운이 좋다고 생각했는데, 1등까지 당첨돼서 기분이 너무 좋”"며 “신종 코로나바이러스 감염증(코로나19) 이후 사업을 정리하고 쉬고 있었는데 큰 도움이 될 것 같다”고 당첨 소감을 밝혔다.'

In [148]:
import MeCab

def count_tokens_korean(word):
    tokenizer = MeCab.Tagger()
    tokens = tokenizer.parse(word).split("\n")
    token_count = len(tokens) - 2  # EOS, 끝의 빈 줄을 제외
    return token_count

word = '''
경기 용인시의 한 복권판매점에서 ‘스피또 1000’ 제71회 복권을 구입한 A씨가 최근 윤 대통령과 커피 마시는 꿈을 꾼 뒤 당첨복권 12장을 교환하러 집 주변에 있는 복권판매점에 방문하여 당첨돼 5억원을 수령했다고 밝혔다. A씨는 “평소 재미 삼아 로또와 스피또 복권을 구매해왔다”며 “최근 윤 대통령과 커피 마시는 꿈을 꿨고, 그 기운을 받아 당첨복권 12장을 교환하러 집 주변에 있는 복권판매점에 방문했다”고 운을 뗐다. 그는 “처음 방문한 판매점에서는 스피또1000 복권이 소진돼서 교환할 수 없었다”며 “두 번째로 간 판매점에 10장이 남아 있어서 10장만 교환했다. 세 번째 판매점에서 남은 2장을 교환하고 집으로 돌아왔다”고 설명했다. 이어 “두 번째 판매점에서 교환한 10장을 다 긁고 마지막 남은 2장을 긁었는데, 5억원이 당첨됐다”며 “남편은 농담하지 말라고 하더라. 처음에는 얼떨떨하고 믿기지 않았는데, 고객센터에 당첨 확인과 방문 예약을 하니 실감이 났다”고 전했다.'''
token_count = count_tokens_korean(word)
print(f"토큰 개수: {token_count}")


토큰 개수: 273


In [142]:
tokenizer = MeCab.Tagger()
tokens = tokenizer.parse('안녕 반가워 다시만나자').split("\n")
tokens

['안녕\tIC,*,T,안녕,*,*,*,*',
 '반가워\tVA+EC,*,F,반가워,Inflect,VA,EC,반갑/VA/*+어/EC/*',
 '다시\tMAG,성분부사|시간부사,F,다시,*,*,*,*',
 '만나\tVV,*,F,만나,*,*,*,*',
 '자\tEC,*,F,자,*,*,*,*',
 'EOS',
 '']

In [18]:
from datetime import datetime

##3. 토크나이저 및 모델 로드
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')


##4. GPU 사용가능 여부 확인 및 Device 설정

if torch.cuda.is_available():
    # GPU를 디바이스로 설정
    device = torch.device("cuda")
else:
    # CPU를 디바이스로 설정
    device = torch.device("cpu")
    
    
    
    

## 5. 모델적용
today = str(datetime.today().strftime("%Y%m%d"))
max_length = 512  # 최대 토큰 수

text = '''
‘스피또 1000’ 제71회 복권 당첨 사연 공개돼    동행복권 홈페이지 갈무리    \xa0  윤석열 대통령과 커피를 마시는 꿈을 꾼 뒤 복권 1등에 당첨됐다는 여성의 사연이 공개됐다.  \xa0  1일 동행복권에 따르면 경기 용인시의 한 복권판매점에서 ‘스피또 1000’ 제71회 복권을 구입한 A씨가\xa0최근 해당 복권 1등에 당첨돼 5억원을 수령했다.  \xa0  A씨는 “평소 재미 삼아 로또와 스피또 복권을 구매해왔다”며 “최근 윤 대통령과 커피 마시는 꿈을 꿨고, 그 기운을 받아 당첨복권 12장을 교환하러 집 주변에 있는 복권판매점에 방문했다”고 운을 뗐다.  \xa0  그는 “처음 방문한 판매점에서는 스피또1000 복권이 소진돼서 교환할 수 없었다”며 “두 번째로 간 판매점에 10장이 남아 있어서 10장만 교환했다. 세 번째 판매점에서 남은 2장을 교환하고 집으로 돌아왔다”고 설명했다.  \xa0  이어 “두 번째 판매점에서 교환한 10장을 다 긁고 마지막 남은 2장을 긁었는데, 5억원이 당첨됐다”며 “남편은 농담하지 말라고 하더라. 처음에는 얼떨떨하고 믿기지 않았는데, 고객센터에 당첨 확인과 방문 예약을 하니 실감이 났다”고 전했다.  \xa0  그러면서 “최근 스피또1000 2등에도 당첨돼서 올해 기운이 좋다고 생각했는데, 1등까지 당첨돼서 기분이 너무 좋”"며 “신종 코로나바이러스 감염증(코로나19) 이후 사업을 정리하고 쉬고 있었는데 큰 도움이 될 것 같다”고 당첨 소감을 밝혔다.
'''
text = text.replace('\n', ' ')
model.to(device)
raw_input_ids = tokenizer.encode(str(text))
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

# 입력 문장 길이 제한
if len(input_ids) > max_length:
    input_ids = input_ids[:max_length]

summary_ids = model.generate(torch.tensor([input_ids]).to(device), repetition_penalty=2.0, num_beams=4, 
                             min_length = 50, max_length=250,  eos_token_id=1)
summary_news = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

summary_news

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


'경기 용인시의 한 복권판매점에서 ‘스피또 1000’ 제71회 복권을 구입한 A씨가 최근 윤 대통령과 커피 마시는 꿈을 꾼 뒤 당첨복권 12장을 교환하러 집 주변에 있는 복권판매점에 방문했다고 밝혔다.'

In [9]:
import pandas as pd
e = pd.read_csv('News_s20230601.csv',dtype= {'press_id': 'object', 'section_id': 'object'})

In [11]:
e[e['section_id']== '110']

,id,headline,writer,url,content,created_date,press_id,section_id
